# Annotating ground truth for object detection

The goal of this notebook is to annotate images which can later be used for training of detection algorithms.

We will configure and run such project in Toloka from scratch.

Performers will be asked to annotate the particular objects in the image.


## The challenge
We have a set of real-life photos of roads:

<table  align="center">
  <tr><td>
    <img src="https://tlk.s3.yandex.net/sdc/photos/0b35956a9afc639a71045f09745096de.jpg"
         alt="Sample road photo"  width="800">
  </td></tr>
  <tr><td align="center">
    <b>Figure 1.</b> Sample road photo
  </td></tr>
</table>

We need to outline every traffic sign. Ultimately, we need to get a set of contours, defined by an array of points, that represent the road signs in each photo. Here’s how it can look like:

<table  align="center">
  <tr><td>
    <img src="./img/segmentation_example.png"
         alt="Example of how road sign detection can be performed"  width="800">
  </td></tr>
  <tr><td align="center">
    <b>Figure 2.</b> Example of how road sign detection can be performed.
  </td></tr>
</table>

In real-world tasks, annotation is usually done with a polygon. We chose to use a rectangular outline to simplify the task so that we can reduce costs and speed things up.

### Detailed task description
In this notebook we will implement Projects 2 and 3 from the [tutorial](https://yandex.com/support/toloka-requester/concepts/image-segmentation-overview.html). Check this link if you want to configure these projects right in the web interface.

We'll skip the first project "Does the image contain a specific object?" from tutorial above, since it's easy to implement using our ["verification project"](https://yandex.com/support/toloka-requester/concepts/image-segmentation-project3.html?lang=en) code.

Here are the two projects we’re going to implement: 
- **Detection project** "[Select an object in the image](https://yandex.com/support/toloka-requester/concepts/image-segmentation-project2.html)" : performers will select image areas that contain a traffic sign.
- **Verification project** "[Are the bounding boxes correct?](https://yandex.com/support/toloka-requester/concepts/image-segmentation-project3.html)" : performers will determine if traffic signs were selected correctly in the images.

[Control tasks](https://yandex.com/support/toloka-requester/concepts/goldenset.html) and [majority vote](https://yandex.com/support/toloka-requester/concepts/mvote.html) aren't used for the detection project, because we can’t expect the area annotations provided by the performers to match each other exactly. Instead, we’ll check detection results in the second project, where a different group of performers will determine whether the traffic signs were annotated correctly or not.

### Set up the environment
First of all, we higly recommend to read [learn the basics example](https://github.com/Toloka/toloka-kit/blob/main/examples/0.getting_started/0.learn_the_basics/) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Toloka/toloka-kit/blob/main/examples/0.getting_started/0.learn_the_basics/learn_the_basics.ipynb) before you start this one to be familiar with the main kinds of entities in Toloka.

In our example we are using the production version of Toloka, but you can also use [Toloka sandbox](https://yandex.com/support/toloka-requester/concepts/sandbox.html). Sandbox specifications of the code are presented in the comments.

Prepare environment and import necessary libraries

In [ ]:
!pip install toloka-kit==0.1.8 # To interact with Toloka API
!pip install ipyplot # To plot images inside Jupyter Notebooks cells
!pip install crowd-kit==0.0.4

import os
import datetime
import time
import logging
import sys

import pandas as pd# To perform data manipulation
import ipyplot

import toloka.client as toloka
import toloka.client.project.template_builder as tb

from crowdkit.aggregation import MajorityVote

logging.basicConfig(
    format='[%(levelname)s] %(name)s: %(message)s',
    level=logging.INFO,
    stream=sys.stdout,
)

Create a toloka-client instance. All API calls will pass through it.

In [ ]:
toloka_client = toloka.TolokaClient(input("Enter your token:"), 'PRODUCTION') # Or switch to 'SANDBOX' 
print(toloka_client.get_requester())

Learn more about [Toloka API](https://yandex.com/dev/toloka/doc/concepts/about.html) and [Toloka Kit](https://toloka.github.io/toloka-kit).


### Review the dataset
The dataset used is collected by Toloka team and distributed under a Creative Commons Attribution 4.0 International license
[![License: CC BY 4.0](https://img.shields.io/badge/License-CC%20BY%204.0-lightgrey.svg)](https://creativecommons.org/licenses/by/4.0/).

Our dataset is just a collection of image URLs.

In [ ]:
!curl https://tlk.s3.yandex.net/dataset/road_signs.tsv --output dataset.tsv

# Load the dataset of links to a pandas DataFrame
dataset = pandas.read_csv('dataset.tsv', sep='\t')

# Plot 5 images from dataset to verify data loading
ipyplot.plot_images(
    [url for url in dataset['image'].sample(n=50)],
    max_images=5,
    img_width=1000
)

---
---
## Create a new detection project

In this project, performers select image areas that contain traffic signs.

The first step is to configure how performers will see the tasks:
* write instructions,
* define the input and output formats.

**Note:** It's important to write clear instructions with examples to make sure the performers do exactly what we want. We also recommend to check the task interface.

In [ ]:
# How performers will see the task
project_interface = toloka.project.view_spec.TemplateBuilderViewSpec(
    config=tb.TemplateBuilder(
        view=tb.ImageAnnotationFieldV1(  # Component that selects areas in images
            image=tb.InputData(path='image'),  # Getter for the input image
            data=tb.OutputData(path='result'),  # Path for writing output data
            shapes={tb.ImageAnnotationFieldV1.Shape.RECTANGLE: True},  # Allow to select only rectangular areas
            validation=tb.RequiredConditionV1(hint='Please select an area')  # At least one area should be selected
        )
    )
)

# You can write instructions and upload them from a file or enter them later in the web interface
# prepared_instruction = open('instruction.html').read().strip()
prepared_instruction = '<b>Outline each traffic sign with a separate bounding box(rectangle).</b>'

# Set up the project
detection_project = toloka.project.Project(
    assignments_issuing_type=toloka.project.Project.AssignmentsIssuingType.AUTOMATED,
    public_name='Outline all traffic signs with bounding boxes',
    public_description='Find and outline all traffic signs with bounding boxes.',
    public_instructions=prepared_instruction,
    # Set up the task: view, input, and output parameters
    task_spec=toloka.project.task_spec.TaskSpec(
        input_spec={'image': toloka.project.field_spec.UrlSpec()},
        output_spec={'result': toloka.project.field_spec.JsonSpec()},
        view_spec=project_interface,
    ),
)

# Call the API to create a new project
detection_project = toloka_client.create_project(detection_project)

### Review your project and check the task interface

Visit the project page to make sure the task interface is working correctly.

To do this:

1. Follow the link in the output above.
2. In the project interface, click **Project actions** on the top right.
3. Click **Preview** in the menu that appears.
4. Click **Change input data**.
5. Insert an image URL (for example, `https://tlk.s3.yandex.net/sdc/photos/0b35956a9afc639a71045f09745096de.jpg`) into the `image` field.
6. Click the **Instructions** button. Make sure the instructions are shown and valid.
7. Try to select multiple areas with a rectangle using **Box annotation tool**.
8. Click **Submit** and then **View responses**.

The result window will appear. Check that your results are in the expected format and that the data is being entered correctly.

<table  align="center">
  <tr><td>
    <img src="./img/segmentation_results_preview.png"
         alt="Task interface"  width="800">
  </td></tr>
  <tr><td align="center">
    <b>Figure 6.</b> What the results window might look like
  </td></tr>
</table>

We strongly recommend to check the task interface and instructions every time you create a project. This helps performers to complete the task correctly and your results to be useful.

**Tip:** Do a trial run with a small amount of data. Make sure that after running the entire pipeline, you get data in the expected format and quality.

### Add custom skills for performers

A skill can describe any characteristic of the performer. Skills are defined by a number from 0 to 100. For example, you can record the percentage of correct responses as a skill. Learn more about [skills](https://yandex.com/support/toloka-requester/concepts/nav.html).

In this project, we'll create two skills: 
- **Detection skill**: Shows the performer completed at least one detection task. We'll later filter out these performers from verification tasks, so that no one can check their own detection. 
- **Verification skill**: How good the current performer is, when compared to others. We'll need this skill later when aggregating the results of the second project.

In [ ]:
detection_skill = next(toloka_client.get_skills(name='Area selection of road signs'), None)
if detection_skill:
    print('Detection skill already exists')
else:
    print('Create new detection skill')
    detection_skill = toloka_client.create_skill(
        name='Area selection of road signs',
        hidden=True,
        public_requester_description={'EN': 'Performer is annotating road signs'},
    )

verification_skill = next(toloka_client.get_skills(name='Detection verification'), None)
if verification_skill:
    print('Verification skill already exists')
else:
    print('Create new verification skill')
    verification_skill = toloka_client.create_skill(
        name='Detection verification',
        hidden=True,
        public_requester_description={'EN': 'How good a performer is at verifying detectionsegmentation tasks'},
    )

### Pool creation for a detection project
A pool is a set of paid tasks sent out for performers.

First, create an instance of the pool and set the basic parameters:
- Payment amount per task.
- Non-automatic acceptance of results.
- Number of tasks performers will see on one page.
- Performer's filter: control who can access this task.

More about configuring and running pools in:
* [Toloka Kit](https://toloka.github.io/toloka-kit/source/toloka.client.pool.html)
* [Help](https://yandex.com/support/toloka-requester/concepts/pool-main.html)
* [Toloka API](https://yandex.com/dev/toloka/doc/concepts/create-pool.html)

In [ ]:
detection_pool = toloka.pool.Pool(
    project_id=detection_project.id,
    private_name='Pool 1',  # Only you can see this information.
    may_contain_adult_content=False,
    will_expire=datetime.datetime.utcnow() + datetime.timedelta(days=365),  # Pool will automatically close after one year
    reward_per_assignment=0.01,     # Set the minimum payment amount for one task page
    auto_accept_solutions=False,    # Only pay the performer for completing the task,
                                    # based on the verification results of the second project.
                                     
    auto_accept_period_day=7,       # Number of days to determine if we'll pay for task completion by this performer or not.
    assignment_max_duration_seconds=60*20,  # Give performers 20 minutes maximum to complete one task page.
    defaults=toloka.pool.Pool.Defaults(
        # We don't need overlapping for detectiondetectionsegmentation tasks, so we set it to 1
        default_overlap_for_new_task_suites=1,
        default_overlap_for_new_tasks=1,
    ),
)

# Set the number of tasks per page
detection_pool.set_mixer_config(real_tasks_count=1, golden_tasks_count=0, training_tasks_count=0)
# Please note that the payment amount specified when creating the pool is the amount the performer receives for completing one page of tasks.
# If you specify 10 tasks per page above, then reward_per_assignment will be paid for completing 10 tasks.

We'll only show our tasks to English-speaking users because the description of the task is in English.
This means that only people who speak English will be able to accept this task.

In [ ]:
detection_pool.filter = toloka.filter.Languages.in_('EN')

**Quality control rules**

View a detailed description of our quality control rules [in documentation](https://yandex.com/support/toloka-requester/concepts/control.html).

Each quality control rule consists of the following:
- **Collector**: How to collect statistics and which metrics can be used in this rule.
- **Condition**: When the rule will be triggered. Under this condition, only parameters that apply to the collector can be used.
- **Action**: What to do if the condition is true.

In [ ]:
# The first rule in this project restricts pool access for performers who often make mistakes
detection_pool.quality_control.add_action(
    collector=toloka.collectors.AcceptanceRate(),
    conditions=[
        # Performer completed more than 2 tasks
        toloka.conditions.TotalAssignmentsCount > 2,
        # And more than 35% of their responses were rejected
        toloka.conditions.RejectedAssignmentsRate > 35,
    ],
    # This action tells Toloka what to do if the condition above is True
    # In our case, we'll restrict access for 15 days
    # Always leave a comment: it may be useful later on
    action=toloka.actions.RestrictionV2(
        scope=toloka.user_restriction.UserRestriction.ALL_PROJECTS,
        duration=15,
        duration_unit='DAYS',
        private_comment='Performer often make mistakes',  # Only you will see this comment
    )
)

# The second useful rule is "Fast responses". It allows us to filter out performers who respond too quickly.
detection_pool.quality_control.add_action(
    # Let's monitor fast submissions for the last 5 completed task pages
    # And define ones that take less than 20 seconds as quick responses.
    collector=toloka.collectors.AssignmentSubmitTime(history_size=5, fast_submit_threshold_seconds=20),
    # If we see more than one fast response, we ban the performer from all our projects for 10 days.
    conditions=[toloka.conditions.FastSubmittedCount > 1],
    action=toloka.actions.RestrictionV2(
        scope=toloka.user_restriction.UserRestriction.ALL_PROJECTS,
        duration=10,
        duration_unit='DAYS',
        private_comment='Fast responses',  # Only you will see this comment
    )
)

# Another rule we use is for automatically updating skills
# We update the detection skill for performers who complete at least one page of tasks from detection pool.
detection_pool.quality_control.add_action(
    collector=toloka.collectors.AnswerCount(),
    # If performer completed at least one task, it sets the new skill to 1
    conditions=[toloka.conditions.AssignmentsAcceptedCount > 0],
    action=toloka.actions.SetSkill(skill_id=detection_skill.id, skill_value=1),
)

# This rule sends rejected assignments (tasks that you rejected) to other performers according to specified parameters.
detection_pool.quality_control.add_action(
    collector=toloka.collectors.AssignmentsAssessment(),
    # Check if a task was rejected
    conditions=[toloka.conditions.AssessmentEvent == toloka.conditions.AssessmentEvent.REJECT],
    # If the condition is True, add 1 to overlap and open the pool
    action=toloka.actions.ChangeOverlap(delta=1, open_pool=True),
)

print('Quality rules count:', len(detection_pool.quality_control.configs))

### Create a pool with all specified conditions

Now we call the Toloka API to finally create a pool in the detection project.

Afterwards, you can check the pool in the web interface. You'll see there aren't any tasks in it. We'll add them later.

In [ ]:
detection_pool = toloka_client.create_pool(detection_pool)

---
---
## Create a new project for verification
In this project, performers will determine if traffic signs were outlined correctly or not.

This will be a standard classification project with only two classes: `OK` and `BAD`. We’ll explicitly define these labels as the output values.

In [ ]:
# Configure task inetrface: how performers will see the task
verification_interface = toloka.project.view_spec.TemplateBuilderViewSpec(
    config=tb.TemplateBuilder(
        view=tb.ListViewV1(  # List of components that should be positioned from top to bottom in the UI
            items=[
                tb.ImageAnnotationFieldV1(  # Image and selected areas to verify
                    image=tb.InputData(path='image'),
                    data=tb.InternalData(path='selection',
                                         default=tb.InputData(path='selection')),  # Use the input field as default value to display the selected areas
                    disabled=True  # Disable adding and deleting areas
                ),
                tb.RadioGroupFieldV1(  # A component for selecting one value out of several options
                    label='Are all traffic signs outlined correctly?',  # Label above the options
                    data=tb.OutputData(path='result'),  # Path for writing output data
                    options=[
                        tb.GroupFieldOption(label='Yes', value='OK'),
                        tb.GroupFieldOption(label='No', value='BAD'),
                    ],
                    validation=tb.RequiredConditionV1()  # Requirement to select one of the options
                )
            ]
        ),
        plugins=[
            tb.HotkeysPluginV1( # Shortcuts for selecting options using the keyboard
                key_1=tb.SetActionV1(data=tb.OutputData(path='result'), payload='OK'),
                key_2=tb.SetActionV1(data=tb.OutputData(path='result'), payload='BAD')
            )
        ]
    )
)

# You can write instructions and upload them from a file or enter them later in the web interface
# prepared_instruction = open('instruction.html').read().strip()
verification_instruction = '''<b>Look at the image and answer the question:</b><br/>
Are all traffic signs outlined correctly?<br/>
If they are, click Yes.<br/>
If they aren't, click No.<br/>
For example, the road signs here are outlined correctly, so the correct answer is Yes.'''

# Set up the project
verification_project = toloka.project.Project(
    assignments_issuing_type=toloka.project.Project.AssignmentsIssuingType.AUTOMATED,
    public_name='Are the traffic signs outlined correctly?',
    public_description='Look at the image and decide whether or not the traffic signs are outlined correctly',
    public_instructions=verification_instruction,
    # Set up the task: view, input, and output parameters
    task_spec=toloka.project.task_spec.TaskSpec(
        input_spec={
            'image': toloka.project.field_spec.UrlSpec(),
            'selection': toloka.project.field_spec.JsonSpec(),
            'assignment_id': toloka.project.field_spec.StringSpec(),
        },
        # Set allowed_values, we'll use smart mixing to get the results of this project
        output_spec={'result': toloka.project.field_spec.StringSpec(allowed_values=['OK', 'BAD'])},
        view_spec=verification_interface,
    ),
)

Call the API to create a new project

In [ ]:
verification_project = toloka_client.create_project(verification_project)

Examine created project in the web interface. To do that:

1. Follow the link above to check the task interface and instructions.

    **Note:** You should see nearly the same interface as in the previous project, only without the ability to select areas.
    It's important to make sure that the annotation results from the first project display correctly in the second one.


2. Open the task **Preview** in the first project.
3. Outline the signs and click **Submit**.
4. Copy the result.
5. Now open the **Preview** of the second project.
6. Click **Change input data** and paste the annotation results in the `selection` field.
7. Click **Apply** and make sure the annotation displays correctly.

### Create and set up a pool in the verification project
We will add filter for this pool: specify performers that don't have the detection skill (as they performed in detection tasks). You can combine multiple conditions using the `&` and `|` operators.

**Note:** we add two quality control rules with the same collector, but with different conditions and actions.

In [ ]:
verification_pool = toloka.pool.Pool(
    project_id=verification_project.id,
    private_name='Pool 1. Road sign verification',  # Only you can see this information.
    may_contain_adult_content=False,
    will_expire=datetime.datetime.utcnow() + datetime.timedelta(days=365),  # Pool will close automatically after one year
    reward_per_assignment=0.01,  # We set the minimum payment amount for one task page
                                 # By default, auto_accept_solutions is on,
                                 # so we'll pay for all the tasks without checking results.
    assignment_max_duration_seconds=60*10,  # Give performers 10 minutes to complete one task page
    defaults=toloka.pool.Pool.Defaults(
        # We need an overlap to compare the performers among themselves,
        # And we need to set an incremental relabeling (dynamic overlap) value less than max_overlap value
        default_overlap_for_new_task_suites=2,
    ),
)

# We'll only show our tasks to English-speaking users because the description of the task is in English.
# We also won't allow our verification tasks to be performed by users who performed detection tasks.
verification_pool.filter = (
    (toloka.filter.Languages.in_('EN')) &
    (toloka.filter.Skill(detection_skill.id) == None)
)

# Set up quality control
# Quality is based on the majority of matching responses from performers who completed the same task.
verification_pool.quality_control.add_action(
    collector=toloka.collectors.MajorityVote(answer_threshold=2),
    # If a performer has 10 or more responses
    # And the responses are correct in less than 50% of cases,
    conditions=[
        toloka.conditions.TotalAnswersCount > 9,
        toloka.conditions.CorrectAnswersRate < 50,
    ],
    # We ban the performer from all our projects for 10 days.
    action=toloka.actions.RestrictionV2(
        scope=toloka.user_restriction.UserRestriction.ALL_PROJECTS,
        duration=10,
        duration_unit='DAYS',
        private_comment=' Doesn\'t match the majority',  # Only you will see this comment
    )
)

# Set up the new skill value using MajorityVote.
# Depending on the percentage of correct responses, we increase the value of the performer's skill.
verification_pool.quality_control.add_action(
    collector=toloka.collectors.MajorityVote(answer_threshold=2, history_size=10),
    conditions=[
        toloka.conditions.TotalAnswersCount > 2,
    ],
    action=toloka.actions.SetSkillFromOutputField(
        skill_id=verification_skill.id,
        from_field='correct_answers_rate',
    ),
)
print('Quality rule count:', len(verification_pool.quality_control.configs))

### Add incremental relabeling and create a pool
For more information about incremental relabeling, check [documentation.](https://yandex.com/support/toloka-requester/concepts/dynamic-overlap.html)


In [ ]:
# Set the task count for one page and turn task shuffling ON to enable incremental relabeling
verification_pool.set_mixer_config(
    real_tasks_count=10,
    golden_tasks_count=0,
    training_tasks_count=0,
    mix_tasks_in_creation_order=True,  # Enable shuffle mode to use incremental relabeling
    force_last_assignment=True,
)
# Create incremental relabeling
verification_pool.set_dynamic_overlap_config(
    type='BASIC',
    max_overlap=5,       # Each task can be completed maximum 5 times
    min_confidence=0.8,  # Percentage, where 100% = 1.0
    answer_weight_skill_id=verification_skill.id,  # Set incremental relabeling by verification skill
    fields=[toloka.pool.DynamicOverlapConfig.Field(name='result')],
)

verification_pool = toloka_client.create_pool(verification_pool)

We need to add incremental relabeling because we'll be using aggregated results from different performers. The overlap will be based on the verification skill.

---
---
## Add tasks to pools and run the projects
At this point, we have configured two projects, and now we can upload the real data that we want to annotate.

In [ ]:
tasks = [
    toloka.task.Task(input_values={'image': url}, pool_id=detection_pool.id)
    for url in dataset['image'].values[:20]
]
# Add tasks to a pool
toloka_client.create_tasks(tasks, toloka.task.CreateTasksParameters(allow_defaults=True))

detection_pool = toloka_client.open_pool(detection_pool.id)

Visit the pool page in the web interface and make sure everything is ok: the number of tasks is correct, the pool is running, and some tasks may already be completed.

<table  align="center">
  <tr><td>
    <img src="./img/segmentation_pool_look.png"
         alt="Pool with tasks"  width="800">
  </td></tr>
  <tr><td align="center">
    <b>Figure 7.</b> How a running pool may look.
  </td></tr>
</table>


Toloka performers work really fast, but they still need time to complete their tasks. We’ll have to wait when they complete all the tasks in the detection pool first.

Remember that you should also review the assignments of detection pool in the web interface. More about [reviewing assignments](https://yandex.com/support/toloka-requester/concepts/offline-accept.html).
You can view the status of the pool in the web interface, but this is not very convenient in a real-life project.

In [ ]:
def wait_pool_for_close(pool):
    sleep_time = 60
    pool = toloka_client.get_pool(pool.id)
    while not pool.is_closed():
        print(
            f'   {datetime.datetime.now().strftime("%H:%M:%S")}\t'
            f'Pool {pool.id} has status {pool.status}.'
        )
        time.sleep(sleep_time)
        pool = toloka_client.get_pool(pool.id)

# Wait for the detection pool
print('\nWaiting for the detection pool to close')
wait_pool_for_close(detection_pool)
print(f'Detection pool {detection_pool.id} is finally closed!')

**Warning:** Do not run the cells below until the detection pool is closed.

When all the tasks in the detection pool have been completed, we can download the results from that pool and prepare data for our verification tasks.

The next step is to run the verification pool.

In [ ]:
def prepare_verification_tasks():
    verification_tasks = []  # Tasks that we will send for verification
    request = toloka.search_requests.AssignmentSearchRequest(
        status=toloka.assignment.Assignment.SUBMITTED,  # Only take completed tasks that haven't been accepted or rejected
        pool_id=detection_pool.id,
    )
    # Create and store new tasks
    for assignment in toloka_client.get_assignments(request):
        verification_tasks.append(
            toloka.task.Task(
                input_values={
                    'image': assignment.tasks[0].input_values['image'],
                    'selection': assignment.solutions[0].output_values['result'],
                    'assignment_id': assignment.id,
                },
                pool_id=verification_pool.id,
            )
        )
    print(f'Generate {len(verification_tasks)} new verification tasks')
    return verification_tasks

def run_verification_pool(verification_tasks):
    verification_tasks_result = toloka_client.create_tasks(
        verification_tasks,
        toloka.task.CreateTasksParameters(allow_defaults=True)
    )
    # We'll store our verification_task-detection_assignments references. We'll need it later.
    task_to_assignment = {}
    for task in verification_tasks_result.items.values():
        task_to_assignment[task.id] = task.input_values['assignment_id']

    # Open the verification pool
    pool = toloka_client.open_pool(verification_pool.id)
    print(f'Verification pool status - {pool.status}')
    return task_to_assignment

# Prepare the tasks
verification_tasks = prepare_verification_tasks()
# Add it to the pool and run the pool
task_to_assignment = run_verification_pool(verification_tasks)

We have just launched our verification pool. Let's wait for it to close.

In [ ]:
print('\nWaiting for verification pool to close')
wait_pool_for_close(verification_pool)
print(f'Verification pool {verification_pool.id} is finally closed!')

If you have overlap in the pool, you need to aggregate the results. We will use [crowd-kit](https://github.com/toloka/crowd-kit) library to perform an aggregation.
In this example, we use MajorityVote. Learn more about [aggregation](https://yandex.com/support/toloka-requester/concepts/result-aggregation.html?lang=en).

Now let’s start the aggregation process. Wait for it to complete and then get the results. We'll use these results to accept or reject the tasks submitted in the detection pool.

In [ ]:
def get_aggregation_results(pool_id, skill_id, submitted_gte=None):
    user_skill = {}
    answers = []
    for assignment in toloka_client.get_assignments(pool_id=pool_id, status='ACCEPTED', submitted_gte=submitted_gte):
        skill = next(toloka_client.get_user_skills(user_id=assignment.user_id, skill_id=skill_id), None)
        skill = skill and skill.value
        user_skill[assignment.user_id] = skill
        for task, solution in zip(assignment.tasks, assignment.solutions):
            answers.append([task.input_values['assignment_id'], solution.output_values['result'], assignment.user_id])
    # Prepare dataframe
    answers_df = pandas.DataFrame(answers, columns=['task', 'label', 'performer'])
    skill_series = pandas.Series(user_skill)
    # Run aggregation
    predicted_answers = MajorityVote().fit_predict(answers_df, skill_series)
    # Get aggregated results
    return predicted_answers

def set_detection_status(verification_results):
    # Reject or accept tasks in the detection pool
    print('Started adding results to detection tasks')
    for assignment_id, result in verification_results.items():
        if result == 'OK':
            toloka_client.accept_assignment(assignment_id, "Well done!")
        else:
            toloka_client.reject_assignment(assignment_id, 'The object wasn\'t selected or was selected incorrectly.')
    print('Finished adding results to detection tasks')


# Aggregation operation
verification_results = get_aggregation_results(verification_pool.id, verification_skill.id)
# Reject or accept tasks in the detection pool
set_detection_status(verification_results)

We may need multiple iterations over our two pools, so we'll write some simple code to do this in a loop. Let's start it and wait.

Depending on the number of images in the detection pool and the time of day, this can take from 10 minutes to almost an hour.

In [ ]:
while True:
    print('\nWaiting for detection pool to close')
    wait_pool_for_close(detection_pool)
    print(f'Detection pool {detection_pool.id} is finally closed!')

    # Preparing tasks
    verification_tasks = prepare_verification_tasks()

    # Make sure all the tasks are done
    if len(verification_tasks) == 0:
        print('All the tasks in our project are done')
        break
    
    verification_submitted_gte = datetime.datetime.utcnow()
    # Add it to the pool and run the pool
    task_to_assignment = run_verification_pool(verification_tasks)

    print('\nWaiting for verification pool to close')
    wait_pool_for_close(verification_pool)
    print(f'Verification pool {verification_pool.id} is finally closed!')

    # Aggregation operation
    verification_results = get_aggregation_results(verification_pool.id, verification_skill.id, verification_submitted_gte)
    # Reject or accept tasks in the detection pool
    set_detection_status(verification_results)


print(f'Results received at {datetime.datetime.now()}')

---
---
## Get the results
Now we can download all the accepted tasks from the detection pool and work with them. In this notebook, we'll only show the detection results.
You can also [download](https://yandex.com/support/toloka-requester/concepts/result-of-eval.html) results as a TSV file from web interface.

In [ ]:
!pip install pillow # To deal with images
!pip install requests # To make HTTP requests
from PIL import Image, ImageDraw
import requests

def get_image(url, selection):
    raw_image = requests.get(url, stream=True).raw
    image = Image.open(raw_image).convert("RGBA")
    regions = Image.new('RGBA', image.size, (255,255,255,0))
    pencil = ImageDraw.Draw(regions)
    for region in selection:
        if region['shape'] != 'rectangle':
            continue
        p1_x = region['left'] * image.size[0]
        p1_y = region['top'] * image.size[1]
        p2_x = (region['left'] + region['width']) * image.size[0]
        p2_y = (region['top'] + region['height']) * image.size[1]
        pencil.rectangle((p1_x, p1_y, p2_x, p2_y), fill =(255, 30, 30, int(255*0.5)))
    image = Image.alpha_composite(image, regions)
    return image

detection_result = {}  # We'll store our result here

In [ ]:
max_images = 2
images = []

if not detection_result:
    request_for_result = toloka.search_requests.AssignmentSearchRequest(
        status=toloka.assignment.Assignment.ACCEPTED,
        pool_id=detection_pool.id,
    )

    for assignment in toloka_client.get_assignments(request_for_result):
        detection_result[assignment.tasks[0].input_values['image']] = assignment.solutions[0].output_values['result']

for i in range(max_images):
    url, selection = detection_result.popitem()
    image = get_image(url, selection)
    images.append(image)

ipyplot.plot_images(
    images,
    max_images=max_images,
    img_width=1000
)

---
---
## Summary

This project consists of the minimum number of settings that will allow you to collect annotated images for your dataset right from Jupyter Notebook.

For your future experiments use Toloka Kit [documentation](https://toloka.github.io/toloka-kit/index.html) and check out other [use cases](https://github.com/Toloka/toloka-kit/tree/main/examples).
